# 6.2 二进制文件格式
* 实现数据的高效二进制格式存储最简单的办法之一是使用Python内置的pickle序列化。pandas对象都有一个用于将数据以pickle格式保存到磁盘上的to_pickle方法：

In [1]:
import pandas as pd
import csv

In [2]:
frame = pd.read_csv('data/examples/ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [3]:
frame.to_pickle('data/frame_pickle')

In [4]:
pd.read_pickle('data/frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 注意：pickle仅建议用于短期存储格式。其原因是很难保证该格式永远是稳定的；今天pickle的对象可能无法被后续版本的库unpickle出来。
* pandas 内置支持两个二进制数据格式：HDF5和MessagePack。

## 使用HDF5文件格式
* HDF5中的HDF指的是层次型数据格式（hierarchical data format）。每个HDF5文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，HDF5支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集，HDF5就是不错的选择，因为它可以高效地分块读写。

# 读取Excel表格数据
* pandas的ExcelFile类或pandas.read_excel函数支持读取存储在Excel 2003（或更高版本）中的表格型数据。这两个工具分别使用扩展包xlrd和openpyxl读取XLS和XLSX文件

In [1]:
import pandas as pd
import numpy as py
xlsx = pd.ExcelFile('data/examples/ex1.xlsx')

In [4]:
#存储在表单中的数据可以read_excel读取到DataFrame
pd.read_excel(xlsx,'Sheet1')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
frame = pd.read_excel('data/examples/ex1.xlsx','Sheet1')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


* 如果要把数据写入到Excel中，首先要创建一个ExcelWriter,然后使用to_excel（）

In [7]:
writer  = pd.ExcelWriter('data/ex2.xlsx')
frame.to_excel(writer,'Sheet1')
writer.save()

* 也可以使用to_excel()

In [8]:
frame.to_excel('data/ex2.xlsx','Sheet2')

# 6.3 Web APIs交互
* 为了搜索最新的30个GitHub上的pandas主题，可以像下面那样操作

In [9]:
import requests
url = "https://api.github.com/repos/pandas-dev/pandas/issues"
resp = requests.get(url)
resp

<Response [200]>

* 响应对象的json()方法会返回一个包含被解析过的JSON字典，加载到一个python对象中：

In [13]:
data = resp.json()
data[0]['url']

'https://api.github.com/repos/pandas-dev/pandas/issues/23507'

* data中的每个元素都是一个包含所有GitHub主题页数据（不包含评论）的字典.我们可以直接传递数据到DataFrame，并提取感兴趣的字段：

In [15]:
issues = pd.DataFrame(data,columns=['number','title','labels','state'])
issues[:5]

,number,title,labels,state
0,23507,Update description of Index._values/values/nda...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
1,23506,DOC: Validate space before colon docstring par...,[],open
2,23505,DOC: Correct docstring formatting for excel re...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
3,23504,Update Index._values docstring for PeriodArray...,"[{'id': 134699, 'node_id': 'MDU6TGFiZWwxMzQ2OT...",open
4,23503,BUG: Timestamp retains frequency of input Time...,"[{'id': 53181044, 'node_id': 'MDU6TGFiZWw1MzE4...",open


# 6.4数据库交互

In [16]:
import sqlite3

In [18]:
query = """
        CREATE TABLE test
        (a VARCHAR(20), b VARCHAR(20),
        c REAL,        d INTEGER
         );"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

In [19]:
con.commit()

* 然后插入几行数据

In [20]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?,?,?,?)"
con.executemany(stmt,data)

* 从表中选取数据时，大部分Python SQL驱动器（PyODBC、psycopg2、MySQLdb、pymssql等）都会返回一个元组列表：

In [21]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

* 将这个元组列表传给DataFrame构造器，但还需要列名（位于光标的description属性中）：

In [22]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [23]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
